In [5]:
import pandas as pd
import os
from llama_index.experimental.query_engine import PandasQueryEngine
from llama_index.core import Settings
from llama_index.llms.openai_like import OpenAILike
from llama_index.core.prompts import PromptTemplate

# ================= 配置区域 =================
# 1. 设置 DeepSeek (和之前一样)
# 记得替换成你的 Key
MY_API_KEY = "sk-bf54fbae108748da859244787c7445d0" 

llm = OpenAILike(
    model="deepseek-chat",
    api_base="https://api.deepseek.com",
    api_key=MY_API_KEY,
    is_chat_model=True,
    temperature=0
)
Settings.llm = llm

# ================= 论文核心算法 =================

# ... (前面的 import 和 LLM 配置保持不变) ...

import pandas as pd
import os
from llama_index.experimental.query_engine import PandasQueryEngine
from llama_index.core import Settings
from llama_index.llms.openai_like import OpenAILike
from llama_index.core.prompts import PromptTemplate

# ================= 配置区域 =================
# 替换你的 Key
MY_API_KEY = "sk-bf54fbae108748da859244787c7445d0" 

llm = OpenAILike(
    model="deepseek-chat",
    api_base="https://api.deepseek.com",
    api_key=MY_API_KEY,
    is_chat_model=True,
    temperature=0
)
Settings.llm = llm

# ================= 论文核心算法 (V3: 强力Prompt版) =================

# ... (前面的 import 和 LLM 配置保持不变) ...

# ... (前面的 import 和配置不变) ...

# ... (前面的 import 和配置不变) ...

class RobustPandasQueryEngine(PandasQueryEngine):
    
    def _query(self, query_bundle):
        # 1. 尝试第一次执行
        response = super()._query(query_bundle)
        response_str = str(response)
        
        # 2. 检查结果是否包含错误
        if "error running" in response_str or "Error message:" in response_str:
            print(f"\n[Agent] ⚠️ 发现潜在错误: {response_str}")
            print(f"[Agent] 🧠 正在启动【自我反思】...")
            
            # 获取列名
            current_columns = list(self._df.columns)
            
            # === V6 核心改进: 更加直白的指令 ===
            retry_prompt_str = (
                "你是一个 Python 表达式生成器。用户的代码报错了。\n"
                "用户问题: {query}\n"
                "报错信息: {error_msg}\n\n"
                "**数据表当前列名**: {columns_info}\n"
                "**任务**：从上面列表中选出最相关的列（例如 'amount'），生成一行清洗并计算的代码。\n\n"
                "**绝对禁止使用占位符！**\n"
                "❌ 错误写法: df['{{column}}'].mean() 或 df['selected_column'].mean()\n"
                "✅ 正确写法: df['amount'].str.replace(r'[$,]', '', regex=True).astype(float).mean()\n\n"
                "**格式约束**：\n"
                "1. 禁止 import，禁止 read_csv。\n"
                "2. 必须是一行 Python 代码，直接使用 df 变量。\n\n"
                "请直接输出代码："
            )
            
            # 格式化 Prompt
            retry_prompt = PromptTemplate(retry_prompt_str).format(
                query=query_bundle.query_str,
                error_msg=response_str,
                columns_info=str(current_columns) 
            )
            
            print(f"[Agent] 🗣️ 发送纠错指令 (已注入列名: {current_columns})...")
            
            # 3. 替换 Prompt 并重试
            original_prompt = self._pandas_prompt
            self._pandas_prompt = PromptTemplate(retry_prompt_str)
            
            try:
                retry_response = super()._query(query_bundle)
                
                # 最终检查
                if "Error message:" in str(retry_response):
                     print(f"[Agent] ❌ 纠错后依然报错: {retry_response}")
                else:
                    print("[Agent] ✅ 纠错完美成功！结果已生成。")
                
                self._pandas_prompt = original_prompt
                return retry_response
                
            except Exception as e:
                print(f"[Agent] ❌ 纠错尝试崩溃: {e}")
                self._pandas_prompt = original_prompt
                return response
        
        else:
            return response

# ================= 实验运行 =================

df = pd.read_csv("sales_data_hidden.csv")
engine = RobustPandasQueryEngine(df=df, verbose=True)

print(">>> [实验开始] 用户提问：计算 amount 的平均值")
response = engine.query("请计算 amount 列的平均值") 

print("\n" + "="*30)
print(f"最终结果: {response}")
print("="*30)

>>> [实验开始] 用户提问：计算 amount 的平均值
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
> Pandas Instructions:
```
df['amount'].mean()
```
> Pandas Output: There was an error running the output as Python code. Error message: Could not convert string '10002000150030002500$9,999' to numeric

[Agent] ⚠️ 发现潜在错误: There was an error running the output as Python code. Error message: Could not convert string '10002000150030002500$9,999' to numeric
[Agent] 🧠 正在启动【自我反思】...
[Agent] 🗣️ 发送纠错指令 (已注入列名: ['id', 'region', 'amount'])...


Traceback (most recent call last):
  File "C:\Users\world\AppData\Roaming\Python\Python312\site-packages\llama_index\experimental\query_engine\pandas\output_parser.py", line 63, in default_output_processor
    output_str = str(safe_eval(module_end_str, global_vars, local_vars))
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\world\AppData\Roaming\Python\Python312\site-packages\llama_index\experimental\exec_utils.py", line 159, in safe_eval
    return eval(__source, _get_restricted_globals(__globals), __locals)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
  File "d:\soft\Lib\site-packages\pandas\core\series.py", line 6549, in mean
    return NDFrame.mean(self, axis, skipna, numeric_only, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\soft\Lib\site-packages\pandas\core\generic.py", line 12420, in mean
    return self._stat_function(
        

INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
> Pandas Instructions:
```
df['amount'].str.replace(r'[$,]', '', regex=True).astype(float).mean()
```
> Pandas Output: 3333.1666666666665
[Agent] ✅ 纠错完美成功！结果已生成。

最终结果: 3333.1666666666665
